In [5]:
%cd ..

/home/batman/ocn/mantaray


In [6]:
"""
Test functionality of squid-py wrapper.
"""

'\nTest functionality of squid-py wrapper.\n'

In [7]:
import pathlib
import squid_py.ocean as ocean_wrapper
from squid_py.utils.web3_helper import convert_to_bytes, convert_to_string, convert_to_text, Web3Helper
import sys
import random
import json
from pprint import pprint
import squid_py.ocean as ocean

In [8]:
import logging

loggers_dict = logging.Logger.manager.loggerDict

logger = logging.getLogger()
logger.handlers = []

# Set level
logger.setLevel(logging.INFO)

# FORMAT = "%(asctime)s - %(levelno)s - %(module)-15s - %(funcName)-15s - %(message)s"
# FORMAT = "%(asctime)s %(levelno)s: %(module)30s %(message)s"
FORMAT = "%(levelno)s - %(module)-15s - %(funcName)-15s - %(message)s"

DATE_FMT = "%Y-%m-%d %H:%M:%S"
DATE_FMT = "%Y-%m-%d %H:%M:%S"
formatter = logging.Formatter(FORMAT, DATE_FMT)

# Create handler and assign
handler = logging.StreamHandler(sys.stderr)
handler.setFormatter(formatter)
logger.handlers = [handler]
logger.info("Logging started")

20 - <ipython-input-8-1b8f9b466b7f> - <module>        - Logging started


In [9]:

# The contract addresses are loaded from file
PATH_CONFIG = pathlib.Path.cwd() / 'config_local.ini'
assert PATH_CONFIG.exists(), "{} does not exist".format(PATH_CONFIG)

ocn = ocean.Ocean(keeper_url='http://0.0.0.0:8545', config_file='config_local.ini')
logging.info("Ocean smart contract node connected ".format())

20 - <ipython-input-9-fe87070c73c5> - <module>        - Ocean smart contract node connected 


In [10]:
ocn.helper.accounts


['0x90F8bf6A479f320ead074411a4B0e7944Ea8c9C1',
 '0xFFcf8FDEE72ac11b5c542428B35EEF5769C409f0',
 '0x22d491Bde2303f2f43325b2108D26f1eAbA1e32b',
 '0xE11BA2b4D45Eaed5996Cd0823791E0C93114882d',
 '0xd03ea8624C8C5987235048901fB614fDcA89b117',
 '0x95cED938F7991cd0dFcb48F0a06a40FA1aF46EBC',
 '0x3E5e9111Ae8eB78Fe1CC3bb8915d5D461F3Ef9A9',
 '0x28a8746e75304c0780E011BEd21C72cD78cd535E',
 '0xACa94ef8bD5ffEE41947b4585a84BdA5a3d3DA6E',
 '0x1dF62f291b2E969fB0849d99D9Ce41e2F137006e']

In [12]:
# By default, 10 wallet addresses are created in Ganache
# A simple wrapper for each address is created to represent a user
# Users are instantiated and listed

class User():
    def __init__(self,num,ocean_obj):
        self.name = 'user' + str(num)
        self.ocean = ocean_obj
        self.address = self.ocean.helper._web3.eth.accounts[num]
        self.balance = self.update_balance()

        logging.debug(self)

    def __str__(self):
        self.update_balance()
        return "{} at {}... with {} token".format(self.name,self.address[0:6],self.balance)

    def update_balance(self):
        self.balance = self.ocean.token.get_token_balance(self.address)

    def request_dev_tokens(self,amount):
        """For development, a user can request free tokens"""
        self.ocean.market.request_tokens(amount, self.address)

    def register_asset(self, dataset):
        # Register this asset on the blockchain
        asset_id = self.ocean.market.register_asset(dataset['base']['name'], dataset['base']['description'],
                                               dataset['base']['price'], self.address)
        assert self.ocean.market.check_asset(asset_id)

        # logging.info("{} registered".format(asset_id.decode("ascii").rstrip()))
        logging.info("registered asset: {}".format(asset_id))


users = list()
for i in range(len(ocn.helper._web3.eth.accounts)):
    user = User(i,ocn)
    user.request_dev_tokens(random.randint(0,100))
    users.append(user)

20 - market          - request_tokens  - Requesting 95 tokens
20 - market          - request_tokens  - Requesting 88 tokens
20 - market          - request_tokens  - Requesting 38 tokens
20 - market          - request_tokens  - Requesting 35 tokens
20 - market          - request_tokens  - Requesting 74 tokens
20 - market          - request_tokens  - Requesting 72 tokens
20 - market          - request_tokens  - Requesting 66 tokens
20 - market          - request_tokens  - Requesting 23 tokens
20 - market          - request_tokens  - Requesting 26 tokens
20 - market          - request_tokens  - Requesting 70 tokens


In [13]:
for u in users: print(u)

user0 at 0x90F8... with 95 token
user1 at 0xFFcf... with 88 token
user2 at 0x22d4... with 38 token
user3 at 0xE11B... with 35 token
user4 at 0xd03e... with 74 token
user5 at 0x95cE... with 72 token
user6 at 0x3E5e... with 66 token
user7 at 0x28a8... with 23 token
user8 at 0xACa9... with 26 token
user9 at 0x1dF6... with 70 token


In [14]:

# The sample asset metadata is stored in a .json file
PATH_ASSET1 = pathlib.Path.cwd() / 'sample_assets' / 'sample1.json'
assert PATH_ASSET1.exists()
with open(PATH_ASSET1) as f:
    dataset = json.load(f)

logging.info("Asset metadata for {}: type={}, price={}".format(dataset['base']['name'],dataset['base']['type'],dataset['base']['price']))

registered_asset = users[0].register_asset(dataset)

asset = ocn.metadata.register_asset(dataset)
assert ocean_provider.metadata.get_asset_ddo(asset['assetId'])['base']['name'] == asset['base']['name']
ocean_provider.metadata.retire_asset(asset['assetId'])

20 - <ipython-input-14-862d26095e5b> - <module>        - Asset metadata for UK Weather information 20111: type=dataset, price=10
20 - market          - register_asset  - registered: b'\xd4\xd4 \xb8\x8aT\xc8\xf8\xe2\xb8\xb3_*\x1esht\xeb\xdd\xd9\x02Y_\x95\xb09\x15 \xe3Ghw'
20 - <ipython-input-12-f35f18a5475e> - register_asset  - registered asset: b'L\xa3i\x9b\x11B\xe4\x02$\x01\xb9F7\x98\xcc\xe9O\xad\xd0\xb8\xe6->\xa31]"!X\xa1\x82\xb8'


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [15]:
asset_ddo = ocn.metadata.get_asset_ddo(dataset['assetId'])
assert ocn.metadata.get_asset_ddo(dataset['assetId'])['base']['name'] == dataset['base']['name']

In [15]:

# import time

# import squid_py.acl as acl
# from squid_py.ocean import Ocean
# from squid_py.utils.web3_helper import convert_to_string

def get_events(event_filter, max_iterations=100, pause_duration=0.1):
    events = event_filter.get_new_entries()
    i = 0
    while not events and i < max_iterations:
        i += 1
        time.sleep(pause_duration)
        events = event_filter.get_new_entries()

    if not events:
        print('no events found in %s events filter.' % str(event_filter))
    return events


def process_enc_token(event):
    # should get accessId and encryptedAccessToken in the event
    print("token published event: %s" % event)


def test_keeper():
    expire_seconds = 9999999999
    asset_price = 100
    ocean = Ocean(host='http://localhost', port=8545, config_path='config_local.ini')
    market = ocean.market
    token = ocean.token
    auth = ocean.auth
    provider_account = ocean.helper.web3.eth.accounts[0]
    consumer_account = ocean.helper.web3.eth.accounts[1]
    assert market.request_tokens(2000, provider_account)
    assert market.request_tokens(2000, consumer_account)

    # 1. Provider register an asset
    asset_id = market.register_asset(json_dict['base']['name'],json_dict['base']['description'], asset_price, provider_account)
    assert market.check_asset(asset_id)
    assert asset_price == market.get_asset_price(asset_id)

    json_dict['assetId'] = ocean.web3.toHex(asset_id)
    ocean.metadata.register_asset(json_dict)
    expiry = int(time.time() + expire_seconds)

    pubprivkey = acl.generate_encryption_keys()
    pubkey = pubprivkey.public_key
    req = auth.concise_contract.initiateAccessRequest(asset_id,
                                                      provider_account,
                                                      pubkey,
                                                      expiry,
                                                      transact={'from': consumer_account})
    receipt = ocean.helper.get_tx_receipt(req)

    send_event = auth.contract.events.AccessConsentRequested().processReceipt(receipt)
    request_id = send_event[0]['args']['_id']

    assert auth.get_order_status(request_id) == 0 or auth.get_order_status(
        request_id) == 1

    # filter_token_published = ocean.helper.watch_event(auth.contract, 'EncryptedTokenPublished', process_enc_token, 0.5,
    #                                                   fromBlock='latest')

    i = 0
    while (auth.get_order_status(request_id) == 1) is False and i < 100:
        i += 1
        time.sleep(0.1)

    assert auth.get_order_status(request_id) == 1

    token.token_approve(ocean.web3.toChecksumAddress(market.address),
                        asset_price,
                        consumer_account)

    buyer_balance_start = token.get_token_balance(consumer_account)
    seller_balance_start = token.get_token_balance(provider_account)
    print('starting buyer balance = ', buyer_balance_start)
    print('starting seller balance = ', seller_balance_start)

    send_payment = market.contract_concise.sendPayment(request_id,
                                                       provider_account,
                                                       asset_price,
                                                       expiry,
                                                       transact={'from': consumer_account, 'gas': 400000})
    receipt = ocean.helper.get_tx_receipt(send_payment)
    print('Receipt: %s' % receipt)

    print('buyer balance = ', token.get_token_balance(consumer_account))
    print('seller balance = ', token.get_token_balance(provider_account))
    ocean.metadata.retire_asset(convert_to_string(asset_id))


    # events = get_events(filter_token_published)
    # assert events
    # assert events[0].args['_id'] == request_id
    # on_chain_enc_token = events[0].args["_encryptedAccessToken"]